# M2S13. 04: Not cluster-based cell annotation

## 04.1. Library and files loading

In [ ]:
library(Seurat)
library(dplyr)

In [ ]:
seurat_obj <- readRDS("01_seurat_obj.Rds")

## 04.2. Module score-based annotation

In [ ]:
cns_markers <- list(
  Astrocytes = c("GFAP", "AQP4", "ALDH1L1", "S100B", "SLC1A3"),
  Astrocytes_Protoplasmic = c("SLC1A2", "ALDH1L1", "GFAP"),
  Astrocytes_Fibrous = c("GFAP", "VIM", "CD44"),
  Oligodendrocyte_Precursors = c("PDGFRA", "CSPG4"),
  Oligodendrocytes_Mature = c("MBP", "MOG", "PLP1"),
  Neurons_Excitatory = c("SLC17A7", "CAMK2A", "GRIN1"),
  Neurons_Inhibitory = c("GAD1", "GAD2", "SLC32A1"),
  Microglia = c("CX3CR1", "P2RY12", "TMEM119", "ITGAM"))

seurat_obj <- AddModuleScore(object = seurat_obj,
  features = cns_markers,
  name = "CNS_Score"
)

score_columns <- grep("CNS_Score", colnames(seurat_obj@meta.data), value = TRUE)
seurat_obj@meta.data$assigned_celltype <- factor(apply(seurat_obj@meta.data[, score_columns], 1, function(x) names(cns_markers)[which.max(x)]))

In [ ]:
cluster_markers <- FindAllMarkers(
    seurat_obj, only.pos = TRUE,
    min.pct = 0.25, logfc.threshold = 0.25,
    group.by = "seurat_clusters"
)

celltype_markers <- FindAllMarkers(
    seurat_obj, only.pos = TRUE,
    min.pct = 0.25, logfc.threshold = 0.25,
    group.by = "assigned_celltype"
)

In [ ]:
top5_clusters <- cluster_markers %>% group_by(cluster) %>% mutate(pctdiff = pct.1-pct.2) %>% top_n(n = 3, wt = pctdiff) 
top5_celltypes <- celltype_markers %>% group_by(cluster) %>% mutate(pctdiff = pct.1-pct.2) %>% top_n(n = 5, wt = pctdiff)

DoHeatmap(subset(seurat_obj, downsample = 30), features = top5_clusters$gene, assay = "RNA", slot = "counts") + NoLegend()
DoHeatmap(subset(seurat_obj, downsample = 30), features = top5_celltypes$gene, group.by = "assigned_celltype", assay = "RNA", slot = "counts") + NoLegend()